In [1]:
# start 27.10.20 20:41

import numpy as np
import random
import math


In [2]:
def genDat(n,l):
    rotL  = lambda x: x[1:] + [x[0]]
    
    rand  = lambda y: map(lambda z: random.choice([0,1]), [0] * y)
    
    x = map(rand, [l] * n)
    y = map(rotL, x)
    
    return zip(x,y)

In [3]:
def mat2(n,m,f):
    return [[f() for j in range(m)] for i in range(n)]

def mat1(n,f):
    return [f() for i in range(n)]

def dot_m_v(m1, v1):
    v = [0] * len(m1)
    for i,em1 in enumerate(m1):
        for j in range(len(em1)):
            v[i] += em1[j] * v1[j]
            
    return v

def dot_mT_v(m1, v1):
    v = [0] * len(m1[0])
    for i,em1 in enumerate(m1):
        for j in range(len(em1)):
            #v[j] += 
            v[j] += m1[i][j] * v1[i]
            
    return v

def sum_v(v1,v2):
    v = [0] * len(v1)
    for i in range(len(v1)):
        v[i] = v1[i] + v2[i]
        
    return v

def minus_v(v1,v2):
    v = [0] * len(v1)
    for i in range(len(v1)):
        v[i] = v1[i] - v2[i]
        
    return v

def mult_v_c(v,c):
    return [v[i] * c for i in range(len(v))]

def sqr_v(v):
    return [v[i] * v[i] for i in range(len(v))]

def mult_m2(m1,c):
    return [[m1[i][j] * c for j in range(len(m1[i]))] for i in range(len(m1))]

def mult_m3(m1,c):
    m = []
    for i in range(len(m1)):
        mx = [[0 for k in range(len(m1[i][0]))] for j in range(len(m1[i]))]
        for j in range(len(m1[i])):
            for k in range(len(m1[i][j])):                
                mx[j][k] = m1[i][j][k] * c
        m += [mx]
        
    return m



def minus_m2(m1,m2):
    m = [[m1[i][j] - m2[i][j] for j in range(len(m1[i]))] for i in range(len(m1))]
        
    return m

def minus_m3(m1,m2):
    m = []
    for i in range(len(m1)):
        mx = [[0 for k in range(len(m1[i][j]))] for j in range(len(m1[i]))]
        for j in range(len(m1[i])):
            for k in range(len(m1[i][j])):                
                mx[j][k] = m1[i][j][k] - m2[i][j][k]
        m += [mx]
        
    return m

def mult_v(v1,v2):
    v = [0] * len(v1)
    for i in range(len(v1)):
        v[i] = v1[i] * v2[i]
        
    return v

def outer_v_v(v1,v2):
    m = [[0 for j in range(len(v2))] for i in range(len(v1))]
    
    for i in range(len(v1)):
        for j in range(len(v2)):
            m[i][j] = v1[i] * v2[j]            
            
    return m
        
def outer_v_v_T(v1,v2):
    m = [[0 for j in range(len(v1))] for i in range(len(v2))]
    
    for i in range(len(v1)):
        for j in range(len(v2)):
            m[j][i] = v1[i] * v2[j]            
            
    return m   


In [4]:
def logsigmoid(z):
    return 1.0/(1.0 + math.exp(-z))

def dlogsigmoid(z):
    return logsigmoid(z) * (1 - logsigmoid(z))



class NN:
    
    def __init__(_, ll, lr = 0.05):
        _.ll  = ll
        _.lr  = lr
        _.fA  = logsigmoid
        _.dfA = dlogsigmoid
        _.w, _.b = _.genWB()
        
    def genWB(_):
        randmat2 = lambda n,m:mat2(n,m,lambda : random.random())
        randmat1 = lambda n  :mat1(n  ,lambda : random.random())
        
        w = [randmat2(nin,nout) for (nin, nout) in zip(_.ll[:-1], _.ll[1:])]
        b = [randmat1(nout)     for nout        in _.ll[1:]]
             
        return w,b
    
    def ff(_, x):
        v  = [None] * len(_.ll)
        vf = [None] * len(_.ll)
        
        w  = _.w
        wl = len(w)
        v[0] = vf[0] = x        
        for i in range(wl):
            #print "ff 1", dot_mT_v(w[i], vf[i])
            #print "ff 2", np.dot(np.transpose(w[i]), vf[i]) 
            v[i+1]  = sum_v(dot_mT_v(w[i], vf[i]), _.b[i])
            vf[i+1] = map(_.fA, v[i+1] )
            
        return v,vf
    
    def fb(_, v, vf, y):
        g  = [None] * len(_.w)
        gw = [None] * len(_.w)
        
        g[-1] = mult_v(minus_v(vf[-1] , y), map(_.dfA, v[-1]))
        
        for i in range(len(_.w) -2 ,-1,-1):
            zz = dot_m_v(_.w[i+1], g[i+1])
            #print "out 1",dot_m_v(_.w[i+1], g[i+1])
            #print "out 2",np.dot(_.w[i+1], g[i+1])
            
            g[i] = mult_v(dot_m_v(_.w[i+1], g[i+1]), map(_.dfA, v[i+1]))
                    
            
        for i in range(len(_.w) -1 ,-1,-1):
            #print "out 1",outer_v_v_T(g[i],vf[i])
            #print "out 2",np.transpose(np.outer(g[i],vf[i]))
            gw[i] = outer_v_v_T(g[i],vf[i])
        
        return gw,g
    
    def trainStep(_,x,y):
        v,vf    = _.ff(x)
        gw,gb   = _.fb(v,vf,y)
        
        _.w     = minus_m3(_.w, mult_m3(gw,_.lr))
        _.b     = minus_m2(_.b, mult_m2(gb,_.lr))
        
        v2,vf2  = _.ff(x)
        
        
        [e1,e2 ] = [_.err(vf[-1],y), _.err(vf2[-1],y)]
        return [e1,e2]
        
    def err(_,out,y):
        return math.sqrt(sum(sqr_v(minus_v(out,y))))
    
    def train(_, data):
        return [_.trainStep(x,y) for (x,y) in data]
    
    def test(_, data):
        for (x,y) in data:
            v,vf    = _.ff(x)
            e       = _.err(vf[-1],y)
            print "x = {}, y = {}, out = {}, e = {}".format(x,y,vf[-1],e)
        
        
            
            

In [7]:
dat = genDat(20000,3)

layers = [[3,3,3], [3,4,5,3]]

for ll in layers:
    print "ll = {}".format(ll)
    nn = NN(ll)
    
    nn.train(dat)
    nn.test(dat[:5])

ll = [3, 3, 3]
x = [1, 0, 0], y = [0, 0, 1], out = [0.0755517730937683, 0.12408355970685356, 0.9516821548200057], e = 0.153099361102
x = [0, 1, 1], y = [1, 1, 0], out = [0.9380857614085276, 0.8898201905920499, 0.05602647549427845], e = 0.138245901559
x = [1, 0, 1], y = [0, 1, 1], out = [0.05319462308482938, 0.9184753758594291, 0.9027082466488279], e = 0.137628549133
x = [0, 0, 0], y = [0, 0, 0], out = [0.09170218543697553, 0.09316917906793042, 0.11390334541188403], e = 0.173389038979
x = [1, 1, 1], y = [1, 1, 1], out = [0.9213511338250894, 0.9159604704552718, 0.8980772652036267], e = 0.15374176578
ll = [3, 4, 5, 3]
x = [1, 0, 0], y = [0, 0, 1], out = [0.29237930533189255, 0.08288475984468523, 0.8721466307466489], e = 0.329699902381
x = [0, 1, 1], y = [1, 1, 0], out = [0.6597158885421963, 0.8863551386290732, 0.14642809468893445], e = 0.38749144241
x = [1, 0, 1], y = [0, 1, 1], out = [0.6201664622074177, 0.6880838326979087, 0.8787297208770576], e = 0.704701792867
x = [0, 0, 0], y = [0, 0